**From a series of application of modern interpolation methods for economics: written by [Mahdi E Kahou](https://sites.google.com/site/mahdiebrahimikahou/about-me)**

# Goal of this notebook
This notebook solves a linear–quadratic (LQ) control problem drawn from Chapter 5 of **[Ljungqvist and Sargent (2018)](https://www.sfu.ca/~kkasa/Recursive_Macroeconomic_Theory_Ljungqvist_Sargent_2018.pdf)**.

## General setup
The general setup of a linear–quadratic (LQ) control problem is given by

$$
\begin{align}
v(x) &= \max_{u}\Bigl\{ -x^\top R x - u^\top Q u 
+ \beta \mathbb{E}\bigl[ v(x') \bigr] \Bigr\} \\
\text{s.t. } \quad x' &= A x + B u + C \epsilon', \\
\qquad x_0 &\text{ is given},
\end{align}
$$

where 
- $x \in \mathbb{R}^n$,
- $u \in \mathbb{R}^m$,  
- $\epsilon' \sim \mathcal{N}(0, I_n)$,  
- $A \in \mathbb{R}^{n \times n}$,  
- $B \in \mathbb{R}^{n \times m}$, and  
- $C \in \mathbb{R}^{n \times n}$.

The reward is **quadratic** in the state $x$ and the control $u$, and the state evolves **linearly**.


The solution takes the form
$$
\begin{align}
v(x) &= -x^\top P x - d, \\
u(x) &= -F x,
\end{align}
$$

where 
- $P \in \mathbb{R}^{n \times n}$,
- $F \in \mathbb{R}^{m \times n}$, and  
- $d \in \mathbb{R}$.

$P$ solves the *algebraic matrix Riccati equation*:

$$
\begin{align*}
P
= R
+ A^\top P A
- A^\top P B \left( Q + B^\top P B \right)^{-1} B^\top P A .
\end{align*}
$$

The optimal feedback matrix $F$ is given by

$$
\begin{align*}
F = \left( Q + B^\top P B \right)^{-1} B^\top P A .
\end{align*}
$$

The constant term $d$ is given by

$$
\begin{align*}
d = \frac{\beta}{1 - \beta} \, \operatorname{trace}\!\left( P C C^\top \right) .
\end{align*}
$$


##Specific Example (Exercise 5.11)

Consider a competitive firm that produces output $y_t$ and sells it at price $p_t$. The firm chooses its production plan to maximize the discounted sum of profits:

$$
\begin{align*}
\sum_{t=0}^{\infty} \beta^t R_t,
\end{align*}
$$

where the period profit (reward) is given by
$$
\begin{align*}
R_t = p_t y_t - \frac{1}{2} \gamma \, (y_{t+1} - y_t)^2,
\end{align*}
$$
with $y_0$ as a known initial output level. 

Here, $\beta \in (0,1)$ represents the discount factor, and $\gamma>0$ captures the cost of changing the production level.

The firm is a price taker, and the market price is determined by the aggregate demand:

$$
\begin{align*}
p_t = \alpha_0 - \alpha_1 Y_t,
\end{align*}
$$

where $\alpha_0, \alpha_1 > 0$, and $Y_t$ denotes total market output, equal to the sum of outputs from $n$ identical firms.

The firm expects the market-wide output to evolve according to

$$
\begin{align*}
Y_{t+1} = h_0 + h_1 Y_t,
\end{align*}
$$

with a known initial condition $Y_0$. At each time $t$, the firm observes both $Y_t$ and its own output $y_t$ before choosing the next period's output $y_{t+1}$.

---
### LQ formulation
In this case 
- $x = \begin{pmatrix} 1\\y\\Y\end{pmatrix}$,
  
- $R = \begin{bmatrix}
0 & -\tfrac{\alpha_0}{2}& 0\\
-\tfrac{\alpha_0}{2} & 0 & \tfrac{\alpha_1}{2}\\
0 & \tfrac{\alpha_1}{2} & 0
\end{bmatrix}$,

- $Q = \tfrac{\gamma}{2}$,
  
- $A=\begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
h_0 & 0 & h_1
\end{bmatrix}$,

- $B = \begin{bmatrix}
0\\1\\0
\end{bmatrix}$, and

- $C = \begin{bmatrix}
0 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0
\end{bmatrix}$.

---
To solve this problem using a neural network, we start from the Euler equation:

### Euler equation

$$
\begin{align}
\gamma u(y,Y) = \beta \big[\gamma u(y',Y') + (\alpha_0 - \alpha_1 Y') \big]
\end{align}
$$

Examining the Euler equation more carefully, we see that the policy function $u(\cdot)$ does not depend on $y$. Therefore, the Euler equation can be simplified to

$$
\begin{align}
\gamma u(Y) = \beta \big[\gamma u(Y') + (\alpha_0 - \alpha_1 Y') \big]
\end{align}
$$


## Deep Learning Implementation

1. **Choose neural networks**  
   Select a neural network to represent the policy $u_\theta(Y)$ and another neural network for the value function $v_\theta(y,Y)$, where $\theta$ denotes the parameters of the networks.

2. **Discretize the state space**  
   We discretize the $(y,Y)$ space as
   $$
   \mathcal{D} = \{y_1, \dots, y_{n_y}\} \times \{Y_1, \dots, Y_{n_Y}\} = \{(y_1,Y_1), \dots, (y_n,Y_n)\},
   $$
   where $n = n_y \cdot n_Y$.

3. **Define the Euler residual**  
   $$
   \ell_\theta^{\text{Euler}}(Y_i) = \gamma u_\theta(Y_i) - \beta \Big[\gamma u_\theta(Y'_i) + (\alpha_0 - \alpha_1 Y'_i) \Big],
   $$
   with $Y'_i = h_0 + h_1 Y_i$.  

4. **Solve the minimization problem**  
   $$
   \min_\theta \frac{1}{n_Y} \sum_{i=1}^{n_Y} \left[\ell_\theta^{\text{Euler}}(Y_i)\right]^2 .
   $$
